# Prepare NetCDF files for WindNinja
### For each *.smet file, retrieve Northing, Easting, T2M, U10M, V10M, and Cloud Cover. 
### Then save to a NetCDF file
### Then merge all NetCDF files into WRF-ARW compliant format in EPSG3031.

In [1]:
import numpy as np
import pandas as pd
import datetime
import matplotlib.pyplot as plt
import re
import xarray as xr
import os

## Function to retrieve met information

In [ ]:
def retrieve_met(file_path):
    # Print file path
    print(file_path)
    
    # Return a data frame with T2M, U10M, and V10M
    df = pd.read_csv(file_path)
    bump = 2
    data_row = np.where(df[df.columns[0]] == '[DATA]')[0][0] + bump
    time = np.loadtxt(file_path, skiprows=data_row, usecols=0, dtype = 'str')
    time = pd.to_datetime(time, format='%Y-%m-%dT%H:%M:%S')
    data = np.loadtxt(file_path, skiprows=data_row, usecols=[1, 5, 6])
    ts = pd.DataFrame(data, index=time)
    ts[ts == -999] = np.nan
    ts.columns = ['T2M [K]', 'U10M [m/s]', 'V10M [m/s]']
    ts.index.name = 'Time'
    
    # Retrieve northing and easting
    easting_line = np.where(df[df.columns[0]].str.startswith("easting"))[0][0]
    easting = float(str(df['SMET 1.1 ASCII'][easting_line]).split()[-1])
    
    northing_line = np.where(df[df.columns[0]].str.startswith("northing"))[0][0]
    northing = float(str(df['SMET 1.1 ASCII'][northing_line]).split()[-1])
    
    # Return output
    return northing, easting, ts

## Testing

In [ ]:
# Gather data
file_path1 = "/scratch/summit/erke2265/SNOWPACK_WAIS/input/meteo/A3D_site_1.smet"
count = 0
stations = []
eastings = []
northings = []
path = "/scratch/summit/erke2265/SNOWPACK_WAIS/input/meteo/"
for file_name in os.listdir(path):
#     if file_name.endswith(".smet") and count < 1:
    if file_name.endswith("34.smet") and count < 1:
        count +=1
        file_path = path + file_name
        print(count)
        northing, easting, df = retrieve_met(file_path)
        stations.append(count)
        northings.append(northing)
        eastings.append(easting)
        if count == 1:
            T2M = np.vstack([df['T2M [K]']])
            U10M = np.vstack([df['U10M [m/s]']])
            V10M = np.vstack([df['V10M [m/s]']])
        else:
            T2M = np.vstack((T2M, [df['T2M [K]']]))
            U10M = np.vstack((U10M, [df['U10M [m/s]']]))
            V10M = np.vstack((V10M, [df['V10M [m/s]']]))

In [ ]:
# Save to xarray
ds_save = xr.Dataset(
    {
        "T2": (("station", "Time"), T2M),
        "U10": (("station", "Time"), U10M),
        "V10": (("station", "Time"), V10M),
        "QCLOUD": (("station", "Time"), np.ones(T2M.shape)),
        "northing":(["station"], northings),
        "easting":(["station"], eastings)
    
    },
    coords={
        "station": (["station"], stations),
        "Time":(("Time"), df.index)
    },
)
ds_save.attrs['TITLE'] = "WRF Proxy"
ds_save.attrs['MAP_PROJ'] = "2"
ds_save

In [ ]:
# Save to NetCDF
!rm WN_forcing.nc
ds_save.to_netcdf("WN_forcing.nc", mode='w')

In [ ]:
# Open NetCDF
ds_open = xr.open_dataset("WN_forcing.nc")
ds_open

In [ ]:
ds_save.close()
ds_open.close()